In [1]:
%run constants.ipynb

In [2]:
import sqlite3
import os

def connect():
    database_path = os.environ["CHESS_ANALYSIS_DB"]
    connection = sqlite3.connect(database_path)
    return connection

In [3]:
from pathlib import Path
import sqlparse

def load_sql(sql_file, placeholder_values = None, show_query = False):
    # Read the query file
    query_file_path = Path("sql") / sql_file
    with open(query_file_path.absolute(), "r") as f:
        query = f.read().replace("\n", " ")

    # Replace placeholders with real values
    if placeholder_values:
        for key, value in placeholder_values.items():
            query = query.replace(f"${key}", value)

    # Show a pretty-printed form of the query
    if show_query:
        print(sqlparse.format(query, reindent=True, keyword_case="upper"))
    return query


In [4]:
import pandas as pd

def load_player_game_ids(connection, name):
    query = load_sql("load_player_game_ids.sql", { "NAME" : name })
    df = pd.read_sql_query(query, connection)
    return df

In [5]:
def load_analysis(connection, game_id, engine):
    # Construct the query
    query = load_sql("load_analysis.sql", {
        "GAME_ID" : str(game_id),
        "ENGINE": engine
    })

    # Read the data
    df = pd.read_sql_query(query, connection)

    # Prepend the move and player indicator
    pos = df.columns.get_loc("halfmove")
    df.insert(pos, "player", df.index.map(lambda i: WHITE if ((1 + i) % 2) > 0 else BLACK))

    pos = df.columns.get_loc("halfmove")
    df.insert(pos, "move", 1 + df.index // 2)

    return df